In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class VisibilitySubSceneDataset(Dataset):
    def __init__(self, obstacle_file, query_file, dim, sub_scene_count):
        """
        Args:
            obstacle_file (str): Path to the file containing obstacle encodings (dim*dim grids).
            query_file (str): Path to the file containing queries (x1, y1, x2, y2, label).
            dim (int): Dimension of the obstacle grid.
            sub_scene_count(int): Number of sub scenes
        """
        # Load the obstacle grids
        self.obstacle_data = self.load_obstacles(obstacle_file, dim, sub_scene_count)
        # Load the queries (x1, y1, x2, y2, label)
        self.query_data = self.load_queries(query_file)
        
        
    def load_obstacles(self, file_path, dim, sub_scene_count):
        """
        Loads obstacle data from a file into memory.
        Args:
            file_path (str): Path to the obstacle file.
            dim (int): Dimension of the grid.
            sub_scene_count(int): Number of sub scenes
        Returns:
            List of numpy arrays (dim, dim) representing the obstacle encodings.
        """
        scenes = []
        with open(file_path, 'r') as file:
            for __ in range(sub_scene_count):
                # Read dim lines to construct the dim*dim grid for each scene
                obstacles = []
                for _ in range(dim):
                    line = list(map(float, file.readline().strip().split()))  # Read each line and split into elements
                    obstacles.append(line)  # Stack all lines to form the grid
                file.readline()
                scenes.append(obstacles)
        return np.array(scenes, dtype=np.float32)
    
    def load_queries(self, file_path):
        """
        Loads query data from a file into memory.
        Args:
            file_path (str): Path to the query file.
        Returns:
            List of tuples (x1, y1, x2, y2, label).
        """
        queries = []
        with open(file_path, 'r') as file:
            for line in file:
                x1, y1, x2, y2, label = map(float, line.split())
                queries.append((x1, y1, x2, y2, label))
        return np.array(queries)
    
    def __len__(self):
        return len(self.query_data)
    
    def __getitem__(self, idx):
        """
        Get a sample from the dataset.
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            A concatenated 1D tensor of obstacle grid and query.
        """
        # Flatten the obstacle grid and convert it to tensor
        obstacle_grid = torch.tensor(self.obstacle_data.reshape(self.obstacle_data.shape[0],-1), dtype=torch.float32)
        
        # Get the query and convert to tensor
        query = torch.tensor(self.query_data[idx][:4], dtype=torch.float32)  # x1, y1, x2, y2
        label = torch.tensor(self.query_data[idx][4], dtype=torch.float32)  # Label
        
        return obstacle_grid, query, label

dim = 64
sub_scene_count = 4

# Initialize the dataset
train_dataset = VisibilitySubSceneDataset("encoding_64x64.txt", "query_64x64.txt", dim, sub_scene_count)


# Create the DataLoader with batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)


In [6]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader

# Define helper function to compute metrics
def get_metrics(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary predictions (0 or 1)
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    specificity = tn / (tn + fp) if tn + fp > 0 else 0  # Specificity = TN / (TN + FP)
    
    return accuracy, recall, precision, f1, specificity

# Define the BinaryClassificationModel
class BinaryClassificationModel(nn.Module):
    def __init__(self, obstacle_dim, embedding_dim, n):
        super(BinaryClassificationModel, self).__init__()

        self.embedding_dim = embedding_dim
        
        # Shared sequential layer for obstacle grid
        self.shared_layer = nn.Sequential(
            nn.Linear(obstacle_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, embedding_dim),
            nn.ReLU(),
            nn.BatchNorm1d(embedding_dim)
        )

        # Final sequential layer for classification
        self.final_layer = nn.Sequential(
            nn.Linear(n * embedding_dim + 4, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, obstacle_grid, query):
        # obstacle_grid shape: (batch, n, obstacle_dim)
        batch_size, n, obstacle_dim = obstacle_grid.size()

        # Process each obstacle grid through the shared layer
        obstacle_grid = obstacle_grid.view(-1, obstacle_dim)  # Shape: (batch * n, obstacle_dim)
        obstacle_embeddings = self.shared_layer(obstacle_grid)  # Shape: (batch * n, embedding_dim)
        obstacle_embeddings = obstacle_embeddings.view(batch_size, n, self.embedding_dim)  # (batch, n, embedding_dim)

        # Flatten embeddings across all obstacles
        obstacle_embeddings = obstacle_embeddings.view(batch_size, -1)  # Shape: (batch, n * embedding_dim)

        # Concatenate query with obstacle embeddings
        combined_input = torch.cat([obstacle_embeddings, query], dim=1)  # Shape: (batch, n * embedding_dim + query_dim)

        # Pass through final classification layer
        output = self.final_layer(combined_input)  # Shape: (batch, 1)
        return output

# Evaluation function to get metrics on validation or test set
def evaluate(model, data_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    total_loss = 0  # To accumulate the loss during evaluation

    with torch.no_grad():
        for obstacle_grids, queries, labels in data_loader:
            obstacle_grids, queries, labels = obstacle_grids.to(device), queries.to(device), labels.to(device)  # Move to device (GPU if available)
            outputs = model(obstacle_grids,queries)

            # Calculate the loss
            loss = criterion(outputs.squeeze(), labels.float())
            total_loss += loss.item()

            all_labels.append(labels.cpu().numpy())
            all_preds.append(outputs.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)

    # Compute metrics over the entire dataset
    accuracy, recall, precision, f1, specificity = get_metrics(all_labels, all_preds)
    
    # Calculate the average loss
    avg_loss = total_loss / len(data_loader)
    
    return avg_loss, accuracy, recall, precision, f1, specificity

# Helper function to train and evaluate the model
def train(model, train_loader, val_loader, num_epochs, lr=1e-3):
    
    model.to(device)  # Move the model to GPU/CPU
    
    # Loss function and optimizer
    criterion = nn.BCELoss()  # Binary Cross-Entropy loss for binary classification
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_f1 = 0  # To track the best model based on validation F1 score

    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set model to training mode
        train_loss = 0
        train_metrics = {"accuracy": [], "recall": [], "precision": [], "f1": [], "specificity": []}

        with tqdm(train_loader, desc="Training", unit="batch") as pbar:
            for obstacle_grids, queries, labels in pbar:
                obstacle_grids, queries, labels = obstacle_grids.to(device), queries.to(device), labels.to(device)  # Move to device (GPU if available)
                
                optimizer.zero_grad()
                outputs = model(obstacle_grids,queries)
                loss = criterion(outputs.squeeze(), labels.float())
                loss.backward()
                optimizer.step()

                # Calculate metrics for the batch
                outputs = outputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                acc, rec, prec, f1, spec = get_metrics(labels, outputs)
                
                train_loss += loss.item()
                train_metrics["accuracy"].append(acc)
                train_metrics["recall"].append(rec)
                train_metrics["precision"].append(prec)
                train_metrics["f1"].append(f1)
                train_metrics["specificity"].append(spec)

                # Update the tqdm progress bar with all metrics
                pbar.set_postfix(
                    loss=train_loss / (pbar.n + 1),
                    accuracy=np.mean(train_metrics["accuracy"]),
                    recall=np.mean(train_metrics["recall"]),
                    precision=np.mean(train_metrics["precision"]),
                    f1=np.mean(train_metrics["f1"]),
                    specificity=np.mean(train_metrics["specificity"])
                )
            
        # Evaluation phase (for training metrics)
        train_loss, train_accuracy, train_recall, train_precision, train_f1, train_specificity = \
            evaluate(model, train_loader, criterion, device)
        
        # Evaluation phase (for validation metrics)
        val_loss, val_accuracy, val_recall, val_precision, val_f1, val_specificity = \
            evaluate(model, val_loader, criterion, device)

        # Display the per-epoch training and validation results
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Recall: {train_recall:.4f}, Precision: {train_precision:.4f}, F1: {train_f1:.4f}, Specificity: {train_specificity:.4f}")
        print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Recall: {val_recall:.4f}, Precision: {val_precision:.4f}, F1: {val_f1:.4f}, Specificity: {val_specificity:.4f}")

        # Save the best model based on validation F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'best_model.pth')

# Check if GPU is available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


model = BinaryClassificationModel(dim*dim,64,sub_scene_count)

# Train and evaluate the model
train(model, train_loader, val_loader, num_epochs=100, lr=1e-3)

model.load_state_dict(torch.load('best_model.pth', weights_only=True))

print("\nEvaluating on Test Set:")
test_loss, test_accuracy, test_recall, test_precision, test_f1, test_specificity = \
    evaluate(model, test_loader, nn.BCELoss(), device)

# Print all metrics for the test set
print(f"Test - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Recall: {test_recall:.4f}, Precision: {test_precision:.4f}, F1: {test_f1:.4f}, Specificity: {test_specificity:.4f}")



Using device: cpu


Training: 100%|██████████| 1/1 [00:00<00:00,  7.05batch/s, accuracy=0.48, f1=0.381, loss=0.749, precision=0.381, recall=0.381, specificity=0.552]


Epoch 1/100
Training - Loss: 0.6785, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 0.6785, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00,  9.60batch/s, accuracy=0.8, f1=0.783, loss=0.654, precision=0.72, recall=0.857, specificity=0.759]

Epoch 2/100

...

Epoch 98/100
Training - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00,  7.76batch/s, accuracy=1, f1=1, loss=0.00127, precision=1, recall=1, specificity=1]


Epoch 99/100
Training - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00,  7.86batch/s, accuracy=1, f1=1, loss=0.00122, precision=1, recall=1, specificity=1]


Epoch 100/100
Training - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 42.0000, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000

Evaluating on Test Set:
Test - Loss: 0.6963, Accuracy: 0.4400, Recall: 1.0000, Precision: 0.4286, F1: 0.6000, Specificity: 0.0345
